# App logic

> Contains BaseChatApp and large language model integration logic. Implements the core functionality indepent of the UI.

In [ ]:
#| default_exp app

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Protocol, runtime_checkable, Generator, List, Dict, Any, Optional, Tuple
import requests
import json
from openai import OpenAI

from gradiochat.config import ModelConfig, Message, ChatAppConfig

In [ ]:
#| export
@runtime_checkable
class LLMClientProtocol(Protocol):
    """Protocol defining the interface for LLM clients"""
    
    def chat_completion(self, messages: List[Message], **kwargs) -> str:
        """Generate a response from the LLM"""
        ...
    
    def chat_completion_stream(self, messages: List[Message], **kwargs) -> Generator[str, None, None]:
        """Generate a streaming response from the LLM"""
        ...

In [ ]:
#| export
class HuggingFaceClient():
    """Client for interacting with HuggingFace models"""
    
    def __init__(self, model_config: ModelConfig):
        """Initialize the client with model configuration"""
        self.model_config = model_config

        # Default to HF Inference API if no base URL is provided
        base_url = model_config.api_base_url or "https//router.huggingface.co/hf-inference/v1"

        self.client = OpenAI(
            base_url=base_url,
            api_key=model_config.api_key or "hf_no_api_key_provided"
        )
    
    def chat_completion(self, 
            messages: List[Message], # List of messages conforming to the Message pydantic dataclass
            **kwargs
            ) -> str:
        """Generate a chat completion from the HuggingFace model"""
        # Convert our Message objects to the format expected by the OpenAI client
        openai_messages = [{"role": msg.role, "content": msg.content} for msg in messages]

        completion = self.client.chat.completions.create(
            model=self.model_config.model_name,
            messages=openai_messages,
            max_tokens=kwargs.get("max_tokens", self.model_config.max_tokens),
            temperature=kwargs.get("temperature", self.model_config.temperature)
        )

        # Extract the generated text
        return completion.choices[0].message.content
    
    def chat_completion_stream(self, messages: List[Message], **kwargs) -> Generator[str, None, None]:
        """Generate a streaming chat completion"""
        # For now, use non-streaming version as a placeholder
        # We'll implement proper streaming later
        result = self.chat_completion(messages, **kwargs)
        yield result

In [ ]:
#| export
def create_llm_client(model_config: ModelConfig) -> LLMClientProtocol:
    """
    Factory function to create an LLM client based on the provider.
    """
    if model_config.provider.lower() == "huggingface":
        return HuggingFaceClient(model_config)
    else:
        raise ValueError(f"Unsupported provider: {model_config.provider}")

In [ ]:
#| export
class BaseChatApp:
    """Base class for creating configurable chat applications with Gradio"""
    
    def __init__(self, config: ChatAppConfig):
        """Initialize the chat application"""
        self.config = config
        self.chat_history = []
        self._load_context()
        self.client = create_llm_client(config.model)
        
    def _load_context(self) -> None:
        """Load context from markdown files"""
        self.context_text = ""
        for file_path in self.config.context_files:
            if file_path.exists() and file_path.is_file():
                with open(file_path, 'r', encoding='utf-8') as f:
                    self.context_text += f.read() + "\n\n"
    
    def prepare_messages(self, user_message: str) -> List[Message]:
        """Prepare the messages for the LLM, including system prompt and chat history"""
        messages = []
        
        # Add system message with prompt and context
        system_content = self.config.system_prompt
        if self.context_text:
            system_content += f"\n\nAdditional information: {self.context_text}"
        
        messages.append(Message(role="system", content=system_content))
        
        # Add chat history
        for user_msg, assistant_msg in self.chat_history:
            messages.append(Message(role="user", content=user_msg))
            messages.append(Message(role="assistant", content=assistant_msg))
        
        # Add current user message
        messages.append(Message(role="user", content=user_message))
        
        return messages
    
    def generate_response(self, user_message: str, **kwargs) -> str:
        """Generate a response to the user message"""
        messages = self.prepare_messages(user_message)
        return self.client.chat_completion(messages, **kwargs)
    
    def generate_stream(self, user_message: str, **kwargs) -> Generator[str, None, None]:
        """Generate a streaming response to the user message"""
        messages = self.prepare_messages(user_message)
        return self.client.chat_completion_stream(messages, **kwargs)

In [ ]:
# Create a test model config
test_config = ModelConfig(
    # model_name="mistralai/Mistral-Nemo-Instruct-2407",
    model_name="mistralai/Mistral-7B-Instruct-v0.3", # "Qwen/QwQ-32B" is another possibility, but with vision you need another messages format
    provider="huggingface",
    api_key_env_var="HF_API_KEY",
    api_base_url="https://router.huggingface.co/hf-inference/v1",
    max_tokens=100,
    temperature=0.7
)

# Create the client
client = create_llm_client(test_config)

test_messages = [
    Message(role="system", content="You are Aurelius Augustinus, helping me to think deeply and be humble and thankfull."),
    Message(role="user", content="Why should I engage with the people around me?")
]
# Test with a simple prompt
try:
    response = client.chat_completion(test_messages)
    print(f"Response received: {response[:100]}...")
except Exception as e:
    print(f"Error: {e}")

# Test with overriden parameters
try:
    print("\nTesting with overridden parameters:")
    response = client.chat_completion(test_messages, max_tokens=50, temperature=0.9)
    print(f"Response: {response[:100]}...")  # Show first 100 chars
except Exception as e:
    print(f"Error: {e}")

Response received: Engaging with the people around you is crucial for several reasons that align with the virtues of hu...

Testing with overridden parameters:
Response: As Aurelius Augustinus, I would encourage you to engage with the people around you for several reaso...


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()